In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [2]:
query = 'SELECT ano, sexo as genero,raca_cor as cor_raca,escolaridade AS grau_instrucao, faixa_etaria,  sigla_uf, count(distinct(id_docente)) as quantidade_pessoas FROM `repositoriodedadosgpsp.Datalake.INEP_docentes`where rede!="privada" group by ano, genero,cor_raca, grau_instrucao, faixa_etaria,  sigla_uf order by ano'

In [3]:
client = bigquery.Client()

In [16]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 103246/103246 [00:06<00:00, 15297.53rows/s]


In [6]:
df.pivot_table(index={'ano'},values='quantidade_pessoas', aggfunc=np.sum)

,quantidade_pessoas
ano,
2009,1533224
2010,1663407
2011,1686326
2012,1726038
2013,1766301
2014,1792557
2015,1789709
2016,1794681
2017,1793158


In [7]:
df.pivot_table(index={'ano'},values='quantidade_pessoas', aggfunc=np.sum)

,quantidade_pessoas
ano,
2009,1533224
2010,1663407
2011,1686326
2012,1726038
2013,1766301
2014,1792557
2015,1789709
2016,1794681
2017,1793158


In [17]:
# Creating dictionary for replace 
dict_esco = {'Pós-Graduação - Mestrado':'Até Pós Graduação ou Mestrado', 'Ensino Médio - Normal/Magistério':'Até Ensino Médio',
       'Ensino Médio':'Até Ensino Médio',
       'Ensino Médio - Normal/Magistério Específico Indígena':'Até Ensino Médio',
       'Fundamental Incompleto':'Até Ensino Fundamental', 'Fundamental Completo':'Até Ensino Fundamental',
       'Ensino Superior Completo':'Até Ensino Superior Completo', 'Pós-Graduação - Doutorado':'Até Doutorado',
       'Pós-Graduação - Especialização':'Até Pós Graduação ou Mestrado',
       'Ensino Superior (Concluído ou em Andamento)':'Até Ensino Superior Completo',
       'Ensino Médio Completo':'Até Ensino Médio'}

In [18]:
df = df.replace({'grau_instrucao':dict_esco})

In [19]:
dict_idade = {'25 a 29 anos':'Entre 18 e 29 anos', '18 a 24 anos':'Entre 18 e 29 anos', '40 a 49 anos':'Entre 30 e 49 anos', '30 a 39 anos':'Entre 30 e 49 anos',
       '50 a 64 anos':'Entre 50 e 64 anos', '65 anos ou mais':'Acima de 64 anos', '15 a 17 anos':'Entre 18 e 29 anos', '10 a 14 anos':'Entre 18 e 29 anos'}

In [20]:
df = df.replace({'faixa_etaria':dict_idade})

In [38]:
## Subindo para o GBQ

In [12]:
df.pivot_table(index={'ano'},values='quantidade_pessoas', aggfunc=np.sum)

,quantidade_pessoas
ano,
2009,1533224
2010,1663407
2011,1686326
2012,1726038
2013,1766301
2014,1792557
2015,1789709
2016,1794681
2017,1793158


In [21]:


schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 #bigquery.SchemaField('tipo_contratacao','STRING',description='Qual o tipo de contratação'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada '),
 bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado'),
 bigquery.SchemaField('faixa_etaria','STRING',description=''),
 #bigquery.SchemaField('rede','STRING',description='Em qual rede essa pessoa atua'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('quantidade_pessoas','INTEGER',description='Quantidade de pessoas')
 ]

In [22]:
dataset_ref = client.dataset('perfil_remuneracao')
table_ref = dataset_ref.table('INEP_professores_ensino_basico')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=49f085c9-8b73-4332-a69b-3ab7cdb25325>